In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import time
import gc
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from collections import Counter


from utils import *

C:\Users\User\anaconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_orig = pd.read_csv("train_data_sila/train_data.csv").set_index('index')


data_train = pd.read_csv("data/synthetic_dataset_for_problem_type_hard.csv").iloc[:, 1:]
data_train = data_train.rename(columns={'result': 'full_text', 'target': 'Точка отказа'})

data_orig['full_text'] = data_orig[['Тема', 'Описание']].apply(lambda x: x[0] + '\n' + x[1], axis=1)
target_column = 'Точка отказа'

C:\Users\User\AppData\Local\Temp\ipykernel_892\2149687356.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  data_orig['full_text'] = data_orig[['Тема', 'Описание']].apply(lambda x: x[0] + '\n' + x[1], axis=1)


In [3]:
data_orig = data_orig[data_orig["Точка отказа"] != 'Консультация']
data_train = data_train[data_train["Точка отказа"] != 'Консультация']

In [4]:
label_counts = data_orig[target_column].value_counts()
single_labels = label_counts[label_counts == 1].index

data_orig_single = data_orig[data_orig[target_column].isin(single_labels)]
data_orig_not_single = data_orig[~data_orig[target_column].isin(single_labels)]

In [5]:
data_orig[target_column].value_counts()

Точка отказа
Материнская плата          43
Блок питания               38
Матрица                    37
Диск                       11
Сервер                      8
Вентилятор                  7
Клавиатура                  7
Диск                        5
Оперативная память          5
Программное обеспечение     4
Wi-fi модуль                4
Wi-fi антенна               4
Корпус                      3
SFP модуль                  1
Аккумулятор                 1
Камера                      1
Динамики                    1
Jack                        1
Name: count, dtype: int64

In [6]:
data_train_orig, data_val_orig = train_test_split(data_orig_not_single, test_size=0.2, random_state=42)
data_val_orig = pd.concat([data_val_orig, data_orig_single], axis=0)

data_train = pd.concat([data_train, data_train_orig], axis=0)

In [7]:
data_train_orig

,Тема,Описание,Тип оборудования,Точка отказа,Серийный номер,full_text
index,,,,,,
171,Не включается дисплей,Не включается дисплей\n,Ноутбук,Материнская плата,C223093407,Не включается дисплей\nНе включается дисплей\n
148,сломался блок питания нотубка Сила НК2-1404 v2...,сломался блок питания нотубка Сила НК2-1404 v2...,Ноутбук,Блок питания,C223101457,сломался блок питания нотубка Сила НК2-1404 v2...
109,Ноутбук СИЛА НК2-1404 // мерцание экрана // C...,"Коллеги, добрый день!\nАнтон, у нас поломка оч...",Ноутбук,Матрица,C223013328,Ноутбук СИЛА НК2-1404 // мерцание экрана // C...
172,Ноутбук Сила не работает порт HDMI,Добрый день_x000D_\nПри подключении внешнего м...,Ноутбук,Материнская плата,C112050125,Ноутбук Сила не работает порт HDMI\nДобрый ден...
44,Ноутбук СИЛА HK1-1425/i5-1135G7/8G/m.2 256G/B3...,Не работает HDMI_x000D_\n_x000D_\nНоутбук СИЛА...,Ноутбук,Материнская плата,С111080030,Ноутбук СИЛА HK1-1425/i5-1135G7/8G/m.2 256G/B3...
...,...,...,...,...,...,...
81,Неисправность сервера // 46909,"Коллеги, добрый день._x000D_\nПросьба принять ...",Сервер,Сервер,D252030013,"Неисправность сервера // 46909\nКоллеги, добры..."
117,не работоспособность BMC,Добрый день._x000D_\n_x000D_\nСервер перестал ...,Сервер,Материнская плата,D253010009,не работоспособность BMC\nДобрый день._x000D_\...
16,Ноутбук СИЛА HK1-1425/ i5-1135G7/8G/m.2 512Gb/...,Ноутбук при загрузке 5-10% отвечает на любые д...,Ноутбук,Материнская плата,С112040045,Ноутбук СИЛА HK1-1425/ i5-1135G7/8G/m.2 512Gb/...


In [13]:
model_id = "cointegrated/LaBSE-en-ru"
# model_id = "ai-forever/ruBert-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModel.from_pretrained(model_id)

# Data

In [16]:
def get_text_embedding(text):
    text_encoded = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors='pt')
    
    with torch.no_grad():
        # emb = torch.nn.functional.normalize(torch.mean(model(**text_encoded)[0], dim=1))
        emb = torch.nn.functional.normalize(model(**text_encoded)[1])
    
    return emb

In [17]:
from torch.utils.data import Dataset, DataLoader


class EvalDataset(Dataset):
    def __init__(self, data, column, temperature_sampling=0.15):

        data = data.reset_index(drop=True)

        self.texts = data['full_text']
        self.labels = data[column]
        # self.label2idx = dict(zip(np.unique(self.labels), np.arange(self.labels.nunique())))
        self.label2idx = dict(zip(np.unique(data_train[column]), np.arange(data_train[column].nunique())))
        self.idx2label = {v:k for k,v in self.label2idx.items()}

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx], self.label2idx[self.labels[idx]]

dataset_train = EvalDataset(data_train, target_column)
dataset_train_for_eval = EvalDataset(data_train, target_column)

dataset_train_orig = EvalDataset(data_train_orig, target_column)
dataset_train_orig_for_eval = EvalDataset(data_train_orig, target_column)

dataset_val = EvalDataset(data_val_orig, target_column)

In [18]:
import pickle
with open("classification_inference/idx2label.pkl", "wb") as f:
    pickle.dump(dataset_train.idx2label, f)

In [20]:
class DataCollator:
    def __init__(self, tokenizer, stage='train'):
        self.tokenizer = tokenizer
        self.stage = stage

    def tokenize(self, text):
        return self.tokenizer(text, padding=True, truncation=True, max_length=512)['input_ids']

    def padding(self, encodings):
        max_length = max(len(encoding) for encoding in encodings)
        attention_masks = torch.stack([torch.tensor([1] * len(enc) + [0] * (max_length - len(enc))) for enc in encodings])
        encodings = torch.stack([torch.tensor(enc + [0] * (max_length - len(enc))) for enc in encodings])
        # encodings = [torch.cat(enc + torch.tensor([0] * (max_length - len(enc))), dim=0) for enc in encodings]

        return {'input_ids': encodings, 'attention_mask': attention_masks}

    def __call__(self, batch_unprocessed):

        if self.stage == 'train':
            batch = {'anchors' : [], 'positives': [], 'negatives' : []}
    
            for (anchor, pos, neg), (label, neg_label) in batch_unprocessed:
                batch['anchors'].append(self.tokenize(anchor))
                batch['positives'].append(self.tokenize(pos))
                batch['negatives'].append(self.tokenize(neg))
    
            batch = {k : self.padding(v) for k, v in batch.items()}
            
        elif self.stage == 'val':
            
            batch = {'encodings': [], 'labels': []}
    
            for text, label in batch_unprocessed:
                batch['encodings'].append(self.tokenize(text))
                batch['labels'].append(label)
    
            batch['encodings'] = self.padding(batch['encodings'])

        else:
            raise ValueError()
        
        return batch

data_collator_train = DataCollator(tokenizer=tokenizer, stage='val')
data_collator_val = DataCollator(tokenizer=tokenizer, stage='val')

In [21]:
dataloader_train = DataLoader(dataset_train, batch_size=8, collate_fn=data_collator_train, shuffle=True)
dataloader_train_for_eval = DataLoader(dataset_train_for_eval, batch_size=32, collate_fn=data_collator_val, shuffle=False)

dataloader_train_orig = DataLoader(dataset_train_orig, batch_size=8, collate_fn=data_collator_train, shuffle=True)
dataloader_train_for_eval_orig = DataLoader(dataset_train_orig_for_eval, batch_size=32, collate_fn=data_collator_val, shuffle=False)

dataloader_val = DataLoader(dataset_val, batch_size=32, collate_fn=data_collator_val, shuffle=False)

# Model

In [23]:
import torch.nn as nn
from torch import optim
from collections import defaultdict
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings("ignore")

In [25]:
criterion = torch.nn.CrossEntropyLoss()

In [26]:
class Model(nn.Module):
    
    def __init__(self, model_id):
        super(Model, self).__init__()
        self.embedder = AutoModel.from_pretrained(model_id)
        self.dropout = nn.Dropout(0.4)
        self.head = nn.Linear(768, 19)


        
    def forward(self, anchors):

        embed = self.embedder(**anchors)[1]
        return self.head(self.dropout(embed))

In [27]:
for b in dataloader_train:
    break

# Training

In [29]:
empty_cache()

model = Model(model_id)

optimizer = optim.Adam(model.parameters(), lr=1e-5)

seed_everything(42)

num_epochs=3
device='cuda'

model.to(device)

Model(
  (embedder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(120138, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_af

In [59]:
def process_batch(batch, device):
    for k in batch.keys():
            for data_type in batch[k].keys():
                batch[k][data_type] = batch[k][data_type].to(device)
    
model.train()
for epoch in tqdm(range(num_epochs), desc="Epochs"):
    running_loss = []
    for batch_idx, batch in tqdm(enumerate(dataloader_train), desc="Training", total=len(dataloader_train)):
        # if batch_idx == 0: break
        batch['encodings'] = {k : v.to(device) for k, v in batch['encodings'].items()}
    
        
        optimizer.zero_grad()
        logits = model(batch['encodings'])
        
        loss = criterion(logits, torch.tensor(batch['labels']).long().to(device))
        loss.backward()
        optimizer.step()
        
        running_loss.append(loss.cpu().detach().numpy())
    print("Epoch: {}/{} - Training Loss: {:.4f}".format(epoch+1, num_epochs, np.mean(running_loss)))

    all_labels_val = []
    all_preds_val = []
    running_loss = []
    for batch in dataloader_val:
        batch['encodings'] = {k : v.to(device) for k, v in batch['encodings'].items()}
    
        
        with torch.no_grad():
            logits = model(batch['encodings'])
        
        loss = criterion(logits, torch.tensor(batch['labels']).long().to(device))
        # 1/0

        all_labels_val.extend(batch['labels'])
        all_preds_val.extend(logits.argmax(dim=-1).tolist())
        
        running_loss.append(loss.cpu().detach().numpy())
    print("Validation Loss: {:.4f}".format(np.mean(running_loss)))

    
    metric = f1_score(all_labels_val, all_preds_val, average='weighted', zero_division=0.0)
    print(f"F1 weighted: {metric}")
    
    print(classification_report(all_labels_val, all_preds_val))
    print('='*50)

Epochs:  33%|████████████                        | 1/3 [00:30<01:01, 30.99s/it]

Epoch: 1/3 - Training Loss: 1.2414
Validation Loss: 1.4349
F1 weighted: 0.5843035810037245
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       1.00      1.00      1.00         1
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         1
           4       1.00      1.00      1.00         1
           5       0.80      0.67      0.73         6
           6       0.00      0.00      0.00         1
           7       0.25      1.00      0.40         1
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         3
          10       0.00      0.00      0.00         1
          11       1.00      1.00      1.00         3
          12       1.00      0.50      0.67         2
          13       0.30      0.43      0.35         7
          14       1.00      0.80      0.89        10
          15       1.00      1.00      1.00 


Epochs:  67%|████████████████████████            | 2/3 [01:01<00:30, 30.52s/it]

Epoch: 2/3 - Training Loss: 0.2552
Validation Loss: 1.3847
F1 weighted: 0.6171893147502904
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       1.00      1.00      1.00         1
           3       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         1
           5       0.67      0.67      0.67         6
           6       0.00      0.00      0.00         1
           7       0.25      1.00      0.40         1
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         3
          10       1.00      1.00      1.00         1
          11       1.00      1.00      1.00         3
          12       1.00      0.50      0.67         2
          13       0.43      0.43      0.43         7
          14       0.82      0.90      0.86        10
          15       0.00      0.00      0.00         1
          16       0.00      0.00      0.00 


Epochs: 100%|████████████████████████████████████| 3/3 [01:31<00:00, 30.44s/it]

Epoch: 3/3 - Training Loss: 0.1622
Validation Loss: 1.3298
F1 weighted: 0.6679218371131079
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       1.00      1.00      1.00         1
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         1
           4       1.00      1.00      1.00         1
           5       1.00      0.67      0.80         6
           6       0.00      0.00      0.00         1
           7       0.33      1.00      0.50         1
           9       1.00      0.33      0.50         3
          10       1.00      1.00      1.00         1
          11       1.00      1.00      1.00         3
          12       1.00      0.50      0.67         2
          13       0.36      0.57      0.44         7
          14       1.00      0.90      0.95        10
          15       0.00      0.00      0.00         1
          16       0.00      0.00      0.00 

In [61]:
num_epochs=12
    
model.train()
for epoch in tqdm(range(num_epochs), desc="Epochs"):
    running_loss = []
    for batch_idx, batch in tqdm(enumerate(dataloader_train_orig), desc="Training", total=len(dataloader_train_orig)):
        # if batch_idx == 0: break
        batch['encodings'] = {k : v.to(device) for k, v in batch['encodings'].items()}
    
        
        optimizer.zero_grad()
        logits = model(batch['encodings'])
        
        loss = criterion(logits, torch.tensor(batch['labels']).long().to(device))
        loss.backward()
        optimizer.step()
        
        running_loss.append(loss.cpu().detach().numpy())
    print("Epoch: {}/{} - Training Loss: {:.4f}".format(epoch+1, num_epochs, np.mean(running_loss)))

    all_labels_val = []
    all_preds_val = []
    running_loss = []
    for batch in dataloader_val:
        batch['encodings'] = {k : v.to(device) for k, v in batch['encodings'].items()}
    
        
        with torch.no_grad():
            logits = model(batch['encodings'])
        
        loss = criterion(logits, torch.tensor(batch['labels']).long().to(device))
        # 1/0

        all_labels_val.extend(batch['labels'])
        all_preds_val.extend(logits.argmax(dim=-1).tolist())
        
        running_loss.append(loss.cpu().detach().numpy())
    print("Validation Loss: {:.4f}".format(np.mean(running_loss)))

    
    metric = f1_score(all_labels_val, all_preds_val, average='weighted', zero_division=0.0)
    print(f"F1 weighted: {metric}")
    
    print(classification_report(all_labels_val, all_preds_val))
    print('='*50)

Epochs:   8%|██▉                                | 1/12 [00:02<00:26,  2.45s/it]

Epoch: 1/12 - Training Loss: 0.2476
Validation Loss: 1.2876
F1 weighted: 0.7134385461501673
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       1.00      1.00      1.00         1
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         1
           4       1.00      1.00      1.00         1
           5       1.00      0.67      0.80         6
           6       0.00      0.00      0.00         1
           7       0.33      1.00      0.50         1
           9       1.00      0.33      0.50         3
          10       1.00      1.00      1.00         1
          11       1.00      1.00      1.00         3
          12       1.00      0.50      0.67         2
          13       0.50      0.71      0.59         7
          14       0.90      0.90      0.90        10
          15       1.00      1.00      1.00         1
          16       0.00      0.00      0.00


Epochs:  17%|█████▊                             | 2/12 [00:04<00:22,  2.25s/it]

Epoch: 2/12 - Training Loss: 0.1954
Validation Loss: 1.4299
F1 weighted: 0.5878527020564324
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       1.00      1.00      1.00         1
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         1
           4       1.00      1.00      1.00         1
           5       1.00      0.67      0.80         6
           6       0.00      0.00      0.00         1
           7       0.25      1.00      0.40         1
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         3
          10       0.00      0.00      0.00         1
          11       1.00      1.00      1.00         3
          12       1.00      0.50      0.67         2
          13       0.36      0.71      0.48         7
          14       1.00      0.70      0.82        10
          15       0.00      0.00      0.00


Epochs:  25%|████████▊                          | 3/12 [00:06<00:19,  2.19s/it]

Epoch: 3/12 - Training Loss: 0.1316
Validation Loss: 1.4370
F1 weighted: 0.5715869496357302
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       1.00      1.00      1.00         1
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         1
           4       1.00      1.00      1.00         1
           5       0.80      0.67      0.73         6
           6       0.00      0.00      0.00         1
           7       0.20      1.00      0.33         1
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         3
          10       0.00      0.00      0.00         1
          11       0.75      1.00      0.86         3
          12       1.00      0.50      0.67         2
          13       0.40      0.29      0.33         7
          14       0.90      0.90      0.90        10
          15       1.00      1.00      1.00


Epochs:  33%|███████████▋                       | 4/12 [00:08<00:17,  2.23s/it]

Epoch: 4/12 - Training Loss: 0.0909
Validation Loss: 1.4277
F1 weighted: 0.5594173441734417
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       1.00      1.00      1.00         1
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         1
           4       1.00      1.00      1.00         1
           5       1.00      0.67      0.80         6
           6       0.00      0.00      0.00         1
           7       0.25      1.00      0.40         1
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         3
          10       0.00      0.00      0.00         1
          11       1.00      1.00      1.00         3
          12       1.00      0.50      0.67         2
          13       0.33      0.43      0.38         7
          14       0.88      0.70      0.78        10
          15       0.33      1.00      0.50


Epochs:  33%|███████████▋                       | 4/12 [00:10<00:20,  2.56s/it]


KeyboardInterrupt: 